# Twitter with twarc
A UCSB original Carpentry workshop

You should have a hashtag_gasprices.jsonl file. We made it for you using the code below.

*This version of the file has most quota-consuming and long-output lines
commented out

In [1]:
# we made this file for you
# ! twarc2 search "#gasprices" > raw_data/hashtag_gasprices.jsonl




In [2]:
# administravia
# upon re-start we need to install twarc2 and other extensions
! pip install twarc-csv
! pip install emoji

# Episode 2


In [3]:
# hashes are comments

print('hello world')

hello world


In [4]:
# BASH commands start with a BANG!
!twarc2 --help

Usage: twarc2 [OPTIONS] COMMAND [ARGS]...

  Collect data from the Twitter V2 API.

Options:
  --consumer-key TEXT         Twitter app consumer key (aka "App Key")
  --consumer-secret TEXT      Twitter app consumer secret (aka "App Secret")
  --access-token TEXT         Twitter app access token for user
                              authentication.
  --access-token-secret TEXT  Twitter app access token secret for user
                              authentication.
  --bearer-token TEXT         Twitter app access bearer token.
  --app-auth / --user-auth    Use application authentication or user
                              authentication. Some rate limits are higher with
                              user authentication, but not all endpoints are
                              supported.  [default: app-auth]
  -l, --log TEXT
  --verbose
  --metadata / --no-metadata  Include/don't include metadata about when and
                              how data was collected.  [default: metadata]
  

In [5]:
#  what libraries will we need to be loading in our notebook?
#  we need to always distinguish between 
#  running BASH vs. running a line of python.

import pandas
import twarc_csv
import textblob
import nltk
import os
import emoji

/opt/conda/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
# this will come into play for ep 8
# !python -m textblob.download_corpora
# nltk.download('stopwords')

In [7]:
# and of course, it's important to know where we are working
# I can send a BASH command from my notebook with a !:
!pwd

/home/jovyan/twarc_run


In [8]:
# you can also do this with Python
os.getcwd()

'/home/jovyan/twarc_run'

In [9]:
# we can change if we need
# os.chdir(".....")

In [10]:
os.getcwd()

'/home/jovyan/twarc_run'

## Running twarc
Let's get the timeline of one of twarc's creators.

In [11]:
# !twarc2 timeline BergisJules > raw_data/bjules.jsonl

### Challenge 1
- Can you find the file called “bjules_flat.jsonl”?
- How many tweets did you get from Bergis? (we can't tell without flattening or looking at the output)
- Download a timeline for a person of your choice. How many tweets did you get? 
- What’s the oldest one?

In [12]:
# !twarc2 timeline ecodatasci > raw_data/ecodatasci.jsonl
! twarc2 flatten raw_data/ecodatasci.jsonl > output_data/ecodatasci_flat.jsonl
! wc output_data/ecodatasci_flat.jsonl

    473  205041 2876068 output_data/ecodatasci_flat.jsonl


A straight harvest using search or stream doesn't need to be flattened 
to do our most basic analysis: wc. Do gas prices here?

## To flatten or not flatten

### Make your jsonl 1 tweet per line
Flattening will let you do our most basic unix-y analysis, turn
timelines into countable lists, and enable you to run twarc1
utilities later on in the workshop

In [13]:
# timeline objects need to be flattened in order to be analyzed as tweets
!twarc2 flatten raw_data/bjules.jsonl output_data/bjules_flat.jsonl

100%|██████████████| Processed 8.96M/8.96M of input file [00:01<00:00, 5.88MB/s]


## Convert to csv

In [14]:
!twarc2 csv raw_data/bjules.jsonl output_data/bjules.csv

100%|██████████████| Processed 8.96M/8.96M of input file [00:02<00:00, 3.62MB/s]

ℹ️
Parsed 3143 tweets objects from 33 lines in the input file.
Wrote 3143 rows and output 74 columns in the CSV.



## When we look at bjules, we really do need to flatten it.

In [15]:
! wc raw_data/bjules.jsonl

     33  845463 9393221 raw_data/bjules.jsonl


33 lines doesn't mean 33 tweets. I suspected there was more there because
I got an error message about hitting a limit of 3200. 

And below, the csv converter tells us there are 3143 tweets.

In [16]:
# convert
!twarc2 csv raw_data/bjules.jsonl output_data/bjules.csv

100%|██████████████| Processed 8.96M/8.96M of input file [00:02<00:00, 3.76MB/s]

ℹ️
Parsed 3143 tweets objects from 33 lines in the input file.
Wrote 3143 rows and output 74 columns in the CSV.



In [17]:
# once I flatten it, my wc will show the correct number
! wc output_data/bjules_flat.jsonl

    3143  1718186 23273101 output_data/bjules_flat.jsonl


In [18]:
# When I did this, I got 3166 tweets (as opposed to the 33 lines that the original file was)
! wc output_data/bjules_flat.jsonl
! wc output_data/bjules.csv

    3143  1718186 23273101 output_data/bjules_flat.jsonl
    3144   579239 11547625 output_data/bjules.csv


The csv is 1 line longer because it has column headers.
twarc2 csv takes flat or unflattened Twitter data files.

### Challenge 2

In [19]:
# commented line is a solution to challenge 1
# !twarc2 timeline ecodatasci > raw_data/ecodatasci.jsonl

!twarc2 flatten raw_data/ecodatasci.jsonl > output_data/ecodatasci_flat.jsonl
!twarc2 csv output_data/ecodatasci_flat.jsonl > output_data/ecodatasci.csv 
ecodatasci_df = pandas.read_csv("output_data/ecodatasci.csv")


# Episode 3: examining tweets
What comes along with a tweet
- Look at one_tweet in Jupyter viewer
- Look at one_tweet with nano
- Look at tweet as csv
- Look at all the entities of a tweet

In [20]:
! wc raw_data/hashtag_gasprices.jsonl

! twarc2 flatten raw_data/hashtag_gasprices.jsonl > output_data/hashtag_gasprices_flat.jsonl

! wc output_data/hashtag_gasprices_flat.jsonl

     108  3346644 36403969 raw_data/hashtag_gasprices.jsonl
   10787  5007559 67146087 output_data/hashtag_gasprices_flat.jsonl


In [21]:
### Let's look at a single tweet as a csv:
!twarc2 flatten raw_data/one_tweet.jsonl output_data/one_tweet_flat.jsonl
!twarc2 csv output_data/one_tweet_flat.jsonl output_data/one_tweet.csv




100%|███████████████| Processed 4.63k/4.63k of input file [00:00<00:00, 607kB/s]
100%|███████████████| Processed 7.09k/7.09k of input file [00:00<00:00, 104kB/s]

ℹ️
Parsed 1 tweets objects from 1 lines in the input file.
Wrote 1 rows and output 74 columns in the CSV.



## first and last tweets

In [22]:
! cat output_data/4_tweets.jsonl

cat: output_data/4_tweets.jsonl: No such file or directory


In [23]:
!head -n 2 'output_data/bjules_flat.jsonl' > 'output_data/4_more_tweets.jsonl'
!tail -n 2 'output_data/bjules_flat.jsonl' >> 'output_data/4_more_tweets.jsonl'

Can we go back further on his timeline by looking
only for Bergis's original content?

Not really--it looks like the limit applies to the search,
not the results. 


But this does tell us that Jules is a prolific re-tweeter and/or replier. 

In [24]:
! wc output_data/bjules_original_flat.jsonl
! wc output_data/bjules_flat.jsonl

wc: output_data/bjules_original_flat.jsonl: No such file or directory
    3143  1718186 23273101 output_data/bjules_flat.jsonl


In [25]:
# save it as a csv so we can easily see the original writings of Jules
!twarc2 csv output_data/bjules_original_flat.jsonl output_data/bjules_original.csv

Usage: twarc2 csv [OPTIONS] [INFILE] [OUTFILE]
Try 'twarc2 csv --help' for help.

Error: Invalid value for '[INFILE]': 'output_data/bjules_original_flat.jsonl': No such file or directory


In [26]:
!head -n 20 'output_data/hashtag_gasprices_flat.jsonl' > 'output_data/20_tweets.jsonl'
!tail -n 20 'output_data/hashtag_gasprices_flat.jsonl' >> 'output_data/20_tweets.jsonl'

# Episode 4

In [27]:
# fishing around for good searches
# you can count without harvesting.
# kittens is an evergreen search. you should always see at lease
# dozens of mentions per hour
!twarc2 counts --text "ucsb"

2022-05-18T19:25:10.000Z - 2022-05-18T20:00:00.000Z: 10
2022-05-18T20:00:00.000Z - 2022-05-18T21:00:00.000Z: 16
2022-05-18T21:00:00.000Z - 2022-05-18T22:00:00.000Z: 14
2022-05-18T22:00:00.000Z - 2022-05-18T23:00:00.000Z: 21
2022-05-18T23:00:00.000Z - 2022-05-19T00:00:00.000Z: 14
2022-05-19T00:00:00.000Z - 2022-05-19T01:00:00.000Z: 13
2022-05-19T01:00:00.000Z - 2022-05-19T02:00:00.000Z: 7
2022-05-19T02:00:00.000Z - 2022-05-19T03:00:00.000Z: 8
2022-05-19T03:00:00.000Z - 2022-05-19T04:00:00.000Z: 11
2022-05-19T04:00:00.000Z - 2022-05-19T05:00:00.000Z: 12
2022-05-19T05:00:00.000Z - 2022-05-19T06:00:00.000Z: 9
2022-05-19T06:00:00.000Z - 2022-05-19T07:00:00.000Z: 6
2022-05-19T07:00:00.000Z - 2022-05-19T08:00:00.000Z: 2
2022-05-19T08:00:00.000Z - 2022-05-19T09:00:00.000Z: 5
2022-05-19T09:00:00.000Z - 2022-05-19T10:00:00.000Z: 4
2022-05-19T10:00:00.000Z - 2022-05-19T11:00:00.000Z: 5
2022-05-19T11:00:00.000Z - 2022-05-19T12:00:00.000Z: 5
2022-05-19T12:00:00.000Z - 2022-05-19T13:00:00.000Z: 2
20

In [28]:
# granularity makes the ouput shorter
# the twitter api is not case sensitive

!twarc2 counts --granularity "day" --text "(UCSB)"
!twarc2 counts --granularity "day" --text "(ucsb)"
!twarc2 counts --granularity "day" --text "(ucsb OR UCSB)"

2022-05-18T19:25:12.000Z - 2022-05-19T00:00:00.000Z: 75
2022-05-19T00:00:00.000Z - 2022-05-20T00:00:00.000Z: 232
2022-05-20T00:00:00.000Z - 2022-05-21T00:00:00.000Z: 342
2022-05-21T00:00:00.000Z - 2022-05-22T00:00:00.000Z: 384
2022-05-22T00:00:00.000Z - 2022-05-23T00:00:00.000Z: 352
2022-05-23T00:00:00.000Z - 2022-05-24T00:00:00.000Z: 306
2022-05-24T00:00:00.000Z - 2022-05-25T00:00:00.000Z: 307
2022-05-25T00:00:00.000Z - 2022-05-25T19:25:12.000Z: 193

Total Tweets: 2,191

2022-05-18T19:25:13.000Z - 2022-05-19T00:00:00.000Z: 75
2022-05-19T00:00:00.000Z - 2022-05-20T00:00:00.000Z: 232
2022-05-20T00:00:00.000Z - 2022-05-21T00:00:00.000Z: 342
2022-05-21T00:00:00.000Z - 2022-05-22T00:00:00.000Z: 384
2022-05-22T00:00:00.000Z - 2022-05-23T00:00:00.000Z: 352
2022-05-23T00:00:00.000Z - 2022-05-24T00:00:00.000Z: 306
2022-05-24T00:00:00.000Z - 2022-05-25T00:00:00.000Z: 307
2022-05-25T00:00:00.000Z - 2022-05-25T19:25:13.000Z: 193

Total Tweets: 2,191

2022-05-18T19:25:15.000Z - 2022-05-19T00:00:00

In [29]:
## hashtags come along with strings. NOT vice versa
!twarc2 counts --text "(Poker OR poker)" --granularity "day"
!twarc2 counts --text "(Poker OR #poker)" --granularity "day"



2022-05-18T19:25:17.000Z - 2022-05-19T00:00:00.000Z: 3,779
2022-05-19T00:00:00.000Z - 2022-05-20T00:00:00.000Z: 18,931
2022-05-20T00:00:00.000Z - 2022-05-21T00:00:00.000Z: 16,047
2022-05-21T00:00:00.000Z - 2022-05-22T00:00:00.000Z: 17,471
2022-05-22T00:00:00.000Z - 2022-05-23T00:00:00.000Z: 12,632
2022-05-23T00:00:00.000Z - 2022-05-24T00:00:00.000Z: 12,400
2022-05-24T00:00:00.000Z - 2022-05-25T00:00:00.000Z: 18,671
2022-05-25T00:00:00.000Z - 2022-05-25T19:25:17.000Z: 12,528

Total Tweets: 112,459

2022-05-18T19:25:18.000Z - 2022-05-19T00:00:00.000Z: 3,778
2022-05-19T00:00:00.000Z - 2022-05-20T00:00:00.000Z: 18,931
2022-05-20T00:00:00.000Z - 2022-05-21T00:00:00.000Z: 16,047
2022-05-21T00:00:00.000Z - 2022-05-22T00:00:00.000Z: 17,472
2022-05-22T00:00:00.000Z - 2022-05-23T00:00:00.000Z: 12,633
2022-05-23T00:00:00.000Z - 2022-05-24T00:00:00.000Z: 12,399
2022-05-24T00:00:00.000Z - 2022-05-25T00:00:00.000Z: 18,671
2022-05-25T00:00:00.000Z - 2022-05-25T19:25:18.000Z: 12,529

Total Tweets: 112

In [30]:
!twarc2 counts --text "(Golf)" --granularity "day"
!twarc2 counts --text "(Basketball)" --granularity "day"
!twarc2 counts --text "(Baseball)" --granularity "day"
!twarc2 counts --text "(Football OR futbol)" --granularity "day"

2022-05-18T19:25:20.000Z - 2022-05-19T00:00:00.000Z: 10,934
2022-05-19T00:00:00.000Z - 2022-05-20T00:00:00.000Z: 55,909
2022-05-20T00:00:00.000Z - 2022-05-21T00:00:00.000Z: 48,832
2022-05-21T00:00:00.000Z - 2022-05-22T00:00:00.000Z: 43,022
2022-05-22T00:00:00.000Z - 2022-05-23T00:00:00.000Z: 46,579
2022-05-23T00:00:00.000Z - 2022-05-24T00:00:00.000Z: 45,258
2022-05-24T00:00:00.000Z - 2022-05-25T00:00:00.000Z: 82,794
2022-05-25T00:00:00.000Z - 2022-05-25T19:25:20.000Z: 45,408

Total Tweets: 378,736

2022-05-18T19:25:22.000Z - 2022-05-19T00:00:00.000Z: 11,838
2022-05-19T00:00:00.000Z - 2022-05-20T00:00:00.000Z: 54,687
2022-05-20T00:00:00.000Z - 2022-05-21T00:00:00.000Z: 63,133
2022-05-21T00:00:00.000Z - 2022-05-22T00:00:00.000Z: 55,623
2022-05-22T00:00:00.000Z - 2022-05-23T00:00:00.000Z: 76,665
2022-05-23T00:00:00.000Z - 2022-05-24T00:00:00.000Z: 64,793
2022-05-24T00:00:00.000Z - 2022-05-25T00:00:00.000Z: 57,518
2022-05-25T00:00:00.000Z - 2022-05-25T19:25:22.000Z: 53,482

Total Tweets: 4

In [31]:
## What's a lot?
!twarc2 counts --text "dog" --granularity "day"
!twarc2 counts --text "cat" --granularity "day"
!twarc2 counts --text "amazon" --granularity "day"
!twarc2 counts --text "right" --granularity "day"
!twarc2 counts --text "good" --granularity "day"


2022-05-18T19:25:28.000Z - 2022-05-19T00:00:00.000Z: 53,394
2022-05-19T00:00:00.000Z - 2022-05-20T00:00:00.000Z: 226,090
2022-05-20T00:00:00.000Z - 2022-05-21T00:00:00.000Z: 217,033
2022-05-21T00:00:00.000Z - 2022-05-22T00:00:00.000Z: 220,185
2022-05-22T00:00:00.000Z - 2022-05-23T00:00:00.000Z: 227,022
2022-05-23T00:00:00.000Z - 2022-05-24T00:00:00.000Z: 211,188
2022-05-24T00:00:00.000Z - 2022-05-25T00:00:00.000Z: 219,349
2022-05-25T00:00:00.000Z - 2022-05-25T19:25:28.000Z: 160,521

Total Tweets: 1,534,782

2022-05-18T19:25:29.000Z - 2022-05-19T00:00:00.000Z: 63,023
2022-05-19T00:00:00.000Z - 2022-05-20T00:00:00.000Z: 355,207
2022-05-20T00:00:00.000Z - 2022-05-21T00:00:00.000Z: 314,843
2022-05-21T00:00:00.000Z - 2022-05-22T00:00:00.000Z: 287,620
2022-05-22T00:00:00.000Z - 2022-05-23T00:00:00.000Z: 299,231
2022-05-23T00:00:00.000Z - 2022-05-24T00:00:00.000Z: 305,764
2022-05-24T00:00:00.000Z - 2022-05-25T00:00:00.000Z: 330,296
2022-05-25T00:00:00.000Z - 2022-05-25T19:25:29.000Z: 245,280


In [32]:
# search for hashtags when you really want hashtags. 
# search for a string returns both text and hashtage (an OR)
# NOT case sensitive
!twarc2 counts --granularity "day" --text "(#UCSB OR UCSB OR ucsb)"
!twarc2 counts --granularity "day" --text "(#ucsb)"
!twarc2 counts --granularity "day" --text "(UCSB)"

2022-05-18T19:25:38.000Z - 2022-05-19T00:00:00.000Z: 74
2022-05-19T00:00:00.000Z - 2022-05-20T00:00:00.000Z: 232
2022-05-20T00:00:00.000Z - 2022-05-21T00:00:00.000Z: 342
2022-05-21T00:00:00.000Z - 2022-05-22T00:00:00.000Z: 384
2022-05-22T00:00:00.000Z - 2022-05-23T00:00:00.000Z: 352
2022-05-23T00:00:00.000Z - 2022-05-24T00:00:00.000Z: 306
2022-05-24T00:00:00.000Z - 2022-05-25T00:00:00.000Z: 307
2022-05-25T00:00:00.000Z - 2022-05-25T19:25:38.000Z: 193

Total Tweets: 2,190

2022-05-18T19:25:40.000Z - 2022-05-19T00:00:00.000Z: 5
2022-05-19T00:00:00.000Z - 2022-05-20T00:00:00.000Z: 11
2022-05-20T00:00:00.000Z - 2022-05-21T00:00:00.000Z: 14
2022-05-21T00:00:00.000Z - 2022-05-22T00:00:00.000Z: 6
2022-05-22T00:00:00.000Z - 2022-05-23T00:00:00.000Z: 3
2022-05-23T00:00:00.000Z - 2022-05-24T00:00:00.000Z: 14
2022-05-24T00:00:00.000Z - 2022-05-25T00:00:00.000Z: 16
2022-05-25T00:00:00.000Z - 2022-05-25T19:25:40.000Z: 5

Total Tweets: 74

2022-05-18T19:25:41.000Z - 2022-05-19T00:00:00.000Z: 74
2022

## Pipeline

In [33]:
## a SFW timeline
# !twarc2 timeline --limit 500 ucsblibrary raw_data/ucsblib_timeline.jsonl

!twarc2 flatten raw_data/ucsblib_timeline.jsonl output_data/ucsblib_timeline_flat.jsonl
!twarc2 csv output_data/ucsblib_timeline_flat.jsonl output_data/ucsblib_timeline_flat.csv
library_timeline_df = pandas.read_csv("output_data/ucsblib_timeline_flat.csv")

100%|██████████████| Processed 1.19M/1.19M of input file [00:00<00:00, 12.1MB/s]
100%|██████████████| Processed 2.57M/2.57M of input file [00:00<00:00, 5.46MB/s]

ℹ️
Parsed 500 tweets objects from 500 lines in the input file.
Wrote 500 rows and output 74 columns in the CSV.



In [34]:
ucsblib_timeline_df = pandas.read_csv("output_data/ucsblib_timeline_flat.csv")

In [35]:
# confirm the dataframe's existance
len(ucsblib_timeline_df)

500

In [36]:
# and view all column headers
list(ucsblib_timeline_df.columns)

['id',
 'conversation_id',
 'referenced_tweets.replied_to.id',
 'referenced_tweets.retweeted.id',
 'referenced_tweets.quoted.id',
 'author_id',
 'in_reply_to_user_id',
 'retweeted_user_id',
 'quoted_user_id',
 'created_at',
 'text',
 'lang',
 'source',
 'public_metrics.like_count',
 'public_metrics.quote_count',
 'public_metrics.reply_count',
 'public_metrics.retweet_count',
 'reply_settings',
 'possibly_sensitive',
 'withheld.scope',
 'withheld.copyright',
 'withheld.country_codes',
 'entities.annotations',
 'entities.cashtags',
 'entities.hashtags',
 'entities.mentions',
 'entities.urls',
 'context_annotations',
 'attachments.media',
 'attachments.media_keys',
 'attachments.poll.duration_minutes',
 'attachments.poll.end_datetime',
 'attachments.poll.id',
 'attachments.poll.options',
 'attachments.poll.voting_status',
 'attachments.poll_ids',
 'author.id',
 'author.created_at',
 'author.username',
 'author.name',
 'author.description',
 'author.entities.description.cashtags',
 'author

In [37]:
ucsblib_timeline_df.head()

,id,conversation_id,referenced_tweets.replied_to.id,referenced_tweets.retweeted.id,referenced_tweets.quoted.id,author_id,in_reply_to_user_id,retweeted_user_id,quoted_user_id,created_at,...,geo.geo.bbox,geo.geo.type,geo.id,geo.name,geo.place_id,geo.place_type,__twarc.retrieved_at,__twarc.url,__twarc.version,Unnamed: 73
0,1528843296411471876,1528843296411471876,NaN,1.527650e+18,NaN,101367986,NaN,483231999.0,NaN,2022-05-23T21:00:21.000Z,...,NaN,NaN,NaN,NaN,NaN,NaN,2022-05-25T18:59:20+00:00,https://api.twitter.com/2/users/101367986/twee...,2.10.4,NaN
1,1527348937379631108,1527348937379631108,NaN,NaN,NaN,101367986,NaN,NaN,NaN,2022-05-19T18:02:18.000Z,...,NaN,NaN,NaN,NaN,NaN,NaN,2022-05-25T18:59:20+00:00,https://api.twitter.com/2/users/101367986/twee...,2.10.4,NaN
2,1527319079727468544,1527319079727468544,NaN,NaN,NaN,101367986,NaN,NaN,NaN,2022-05-19T16:03:39.000Z,...,NaN,NaN,NaN,NaN,NaN,NaN,2022-05-25T18:59:20+00:00,https://api.twitter.com/2/users/101367986/twee...,2.10.4,NaN
3,1526956500354314241,1526956500354314241,NaN,NaN,NaN,101367986,NaN,NaN,NaN,2022-05-18T16:02:53.000Z,...,NaN,NaN,NaN,NaN,NaN,NaN,2022-05-25T18:59:20+00:00,https://api.twitter.com/2/users/101367986/twee...,2.10.4,NaN
4,1524072186599911425,1524072186599911425,NaN,NaN,NaN,101367986,NaN,NaN,NaN,2022-05-10T17:01:39.000Z,...,NaN,NaN,NaN,NaN,NaN,NaN,2022-05-25T18:59:20+00:00,https://api.twitter.com/2/users/101367986/twee...,2.10.4,NaN


In [38]:
ucsblib_timeline_df.tail()

,id,conversation_id,referenced_tweets.replied_to.id,referenced_tweets.retweeted.id,referenced_tweets.quoted.id,author_id,in_reply_to_user_id,retweeted_user_id,quoted_user_id,created_at,...,geo.geo.bbox,geo.geo.type,geo.id,geo.name,geo.place_id,geo.place_type,__twarc.retrieved_at,__twarc.url,__twarc.version,Unnamed: 73
495,1350139793909301248,1350139793909301248,NaN,1.350138e+18,NaN,101367986,NaN,2.542162e+09,NaN,2021-01-15T17:56:25.000Z,...,NaN,NaN,NaN,NaN,NaN,NaN,2022-05-25T18:59:24+00:00,https://api.twitter.com/2/users/101367986/twee...,2.10.4,NaN
496,1350104526250942468,1350104526250942468,NaN,NaN,NaN,101367986,NaN,NaN,NaN,2021-01-15T15:36:17.000Z,...,NaN,NaN,NaN,NaN,NaN,NaN,2022-05-25T18:59:24+00:00,https://api.twitter.com/2/users/101367986/twee...,2.10.4,NaN
497,1349772447541653505,1349217632264744961,1.349763e+18,NaN,NaN,101367986,1.285968e+18,NaN,NaN,2021-01-14T17:36:43.000Z,...,NaN,NaN,NaN,NaN,NaN,NaN,2022-05-25T18:59:24+00:00,https://api.twitter.com/2/users/101367986/twee...,2.10.4,NaN
498,1349771964525674500,1349502619018907650,1.349503e+18,NaN,NaN,101367986,1.302718e+08,NaN,NaN,2021-01-14T17:34:48.000Z,...,NaN,NaN,NaN,NaN,NaN,NaN,2022-05-25T18:59:24+00:00,https://api.twitter.com/2/users/101367986/twee...,2.10.4,NaN
499,1349771609976893440,1349771609976893440,NaN,1.349503e+18,NaN,101367986,NaN,1.302718e+08,NaN,2021-01-14T17:33:24.000Z,...,NaN,NaN,NaN,NaN,NaN,NaN,2022-05-25T18:59:24+00:00,https://api.twitter.com/2/users/101367986/twee...,2.10.4,NaN


In [39]:
#grab only specified column
ucsblib_timeline_df['public_metrics.retweet_count']

0      2
1      0
2      0
3      0
4      2
      ..
495    1
496    2
497    0
498    0
499    1
Name: public_metrics.retweet_count, Length: 500, dtype: int64

In [40]:
sort_by_rt = ucsblib_timeline_df.sort_values('public_metrics.retweet_count', ascending=False)

#the first tweet from the sorted dataframe
most_rt = sort_by_rt.head(1)

#output the text of the most retweeted tweet
most_rt['text']

166    For the first time in history, sound recording...
Name: text, dtype: object

In [41]:
sort_by_rt['public_metrics.retweet_count'].head(20)

166    164
286    114
439     84
283     65
289     54
173     29
199     26
450     24
313     23
436     22
441     21
449     21
200     18
416     18
407     17
133     16
13      15
315     15
382     14
453     13
Name: public_metrics.retweet_count, dtype: int64

## final challenge: Cats of Instagram
Let’s make a bigger datafile. Harvest 5000 tweets that use the hashtag “catsofinstagram” and put the dataset through the pipeline to answer the following questions:

- Did you get exactly 5000?
- How far back in time did you get?
- What is the most re-tweeted recent tweet on #catsofinstagram?
- Which person has the most number of followers in your dataset?
- Is it really a person?

In [42]:
# !twarc2 search --limit 500 "#catsofinstagram" raw_data/hashtagcats.jsonl
!twarc2 flatten raw_data/hashtagcats.jsonl output_data/hashtagcats_flat.jsonl
!twarc2 csv raw_data/hashtagcats.jsonl > output_data/hashtagcats.csv
hashtagcats_df = pandas.read_csv("output_data/hashtagcats.csv")
! wc output_data/hashtagcats.csv
hashtagcats_df["created_at"].head()

100%|██████████████| Processed 1.84M/1.84M of input file [00:00<00:00, 11.0MB/s]
    600  118388 2181953 output_data/hashtagcats.csv


0    2022-05-24T00:17:07.000Z
1    2022-05-24T00:14:06.000Z
2    2022-05-24T00:14:06.000Z
3    2022-05-24T00:12:02.000Z
4    2022-05-24T00:11:58.000Z
Name: created_at, dtype: object

In [43]:
hashtagcats_df["created_at"].tail()

594    2022-05-23T15:27:12.000Z
595    2022-05-23T15:24:35.000Z
596    2022-05-23T15:22:56.000Z
597    2022-05-23T15:21:13.000Z
598    2022-05-23T15:20:22.000Z
Name: created_at, dtype: object

In [44]:
hashtagcats_df

,id,conversation_id,referenced_tweets.replied_to.id,referenced_tweets.retweeted.id,referenced_tweets.quoted.id,author_id,in_reply_to_user_id,retweeted_user_id,quoted_user_id,created_at,...,geo.geo.bbox,geo.geo.type,geo.id,geo.name,geo.place_id,geo.place_type,__twarc.retrieved_at,__twarc.url,__twarc.version,Unnamed: 73
0,1528892818105815040,1528892818105815040,NaN,1.517710e+18,NaN,1128200401,NaN,3.958303e+09,NaN,2022-05-24T00:17:07.000Z,...,NaN,NaN,NaN,NaN,NaN,NaN,2022-05-24T00:20:37+00:00,https://api.twitter.com/2/tweets/search/recent...,2.10.4,NaN
1,1528892058274091012,1528892058274091012,NaN,NaN,NaN,749661007,NaN,NaN,NaN,2022-05-24T00:14:06.000Z,...,NaN,NaN,NaN,NaN,NaN,NaN,2022-05-24T00:20:37+00:00,https://api.twitter.com/2/tweets/search/recent...,2.10.4,NaN
2,1528892056524972033,1528892056524972033,NaN,1.528889e+18,NaN,1088162497725648908,NaN,9.486312e+17,NaN,2022-05-24T00:14:06.000Z,...,NaN,NaN,NaN,NaN,NaN,NaN,2022-05-24T00:20:37+00:00,https://api.twitter.com/2/tweets/search/recent...,2.10.4,NaN
3,1528891537995796481,1528891537995796481,NaN,NaN,NaN,444109960,NaN,NaN,NaN,2022-05-24T00:12:02.000Z,...,NaN,NaN,NaN,NaN,NaN,NaN,2022-05-24T00:20:37+00:00,https://api.twitter.com/2/tweets/search/recent...,2.10.4,NaN
4,1528891522271150081,1528891522271150081,NaN,1.528845e+18,NaN,1145235467601756161,NaN,1.350456e+18,NaN,2022-05-24T00:11:58.000Z,...,NaN,NaN,NaN,NaN,NaN,NaN,2022-05-24T00:20:37+00:00,https://api.twitter.com/2/tweets/search/recent...,2.10.4,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
594,1528759456552693760,1528759456552693760,NaN,1.528738e+18,NaN,774243018304057345,NaN,1.479934e+18,NaN,2022-05-23T15:27:12.000Z,...,NaN,NaN,NaN,NaN,NaN,NaN,2022-05-24T00:20:41+00:00,https://api.twitter.com/2/tweets/search/recent...,2.10.4,NaN
595,1528758800408121344,1528758800408121344,NaN,1.528635e+18,NaN,911449457824108545,NaN,1.251569e+18,NaN,2022-05-23T15:24:35.000Z,...,NaN,NaN,NaN,NaN,NaN,NaN,2022-05-24T00:20:41+00:00,https://api.twitter.com/2/tweets/search/recent...,2.10.4,NaN
596,1528758385394733059,1528758385394733059,NaN,1.528529e+18,NaN,1425297926981488643,NaN,1.524979e+18,NaN,2022-05-23T15:22:56.000Z,...,NaN,NaN,NaN,NaN,NaN,NaN,2022-05-24T00:20:41+00:00,https://api.twitter.com/2/tweets/search/recent...,2.10.4,NaN
597,1528757951221096448,1528757951221096448,NaN,NaN,NaN,722125004054388736,NaN,NaN,NaN,2022-05-23T15:21:13.000Z,...,NaN,NaN,NaN,NaN,NaN,NaN,2022-05-24T00:20:41+00:00,https://api.twitter.com/2/tweets/search/recent...,2.10.4,NaN


In [45]:
list(hashtagcats_df.columns)

['id',
 'conversation_id',
 'referenced_tweets.replied_to.id',
 'referenced_tweets.retweeted.id',
 'referenced_tweets.quoted.id',
 'author_id',
 'in_reply_to_user_id',
 'retweeted_user_id',
 'quoted_user_id',
 'created_at',
 'text',
 'lang',
 'source',
 'public_metrics.like_count',
 'public_metrics.quote_count',
 'public_metrics.reply_count',
 'public_metrics.retweet_count',
 'reply_settings',
 'possibly_sensitive',
 'withheld.scope',
 'withheld.copyright',
 'withheld.country_codes',
 'entities.annotations',
 'entities.cashtags',
 'entities.hashtags',
 'entities.mentions',
 'entities.urls',
 'context_annotations',
 'attachments.media',
 'attachments.media_keys',
 'attachments.poll.duration_minutes',
 'attachments.poll.end_datetime',
 'attachments.poll.id',
 'attachments.poll.options',
 'attachments.poll.voting_status',
 'attachments.poll_ids',
 'author.id',
 'author.created_at',
 'author.username',
 'author.name',
 'author.description',
 'author.entities.description.cashtags',
 'author

In [46]:
# what dataframes do we have at this point?
%whos DataFrame

Variable              Type         Data/Info
--------------------------------------------
ecodatasci_df         DataFrame                          id <...>\n[473 rows x 74 columns]
hashtagcats_df        DataFrame                          id <...>\n[599 rows x 74 columns]
library_timeline_df   DataFrame                          id <...>\n[500 rows x 74 columns]
most_rt               DataFrame                          id <...>\n\n[1 rows x 74 columns]
sort_by_rt            DataFrame                          id <...>\n[500 rows x 74 columns]
ucsblib_timeline_df   DataFrame                          id <...>\n[500 rows x 74 columns]


# Episode 5: Ethics & Twitter

In [47]:
# what dataframes do we have?
# this version is more succinct, but not nicely formated
%who DataFrame


ecodatasci_df	 hashtagcats_df	 library_timeline_df	 most_rt	 sort_by_rt	 ucsblib_timeline_df	 


In [48]:
# our first full-text analysis
# a list of words with TextBlob

# first we need to munge the data. remember from:
# list(library_df.columns)
# the tweet is library_df['text']

# TextBlob has its own data format.

# break tweets test column into a list, 
# then .join into one long string 
library_string = ' '.join(library_timeline_df['text'].tolist())
# turn the string into a blob
library_blob = textblob.TextBlob(library_string)


In [49]:
# This produces a mess if we output it. 
# (library_blob)

In [50]:
# Let's count the words and sort by their frequency of use:
library_freq = library_blob.word_counts
library_sorted_freq = sorted(library_freq.items(), 
                             key = lambda kv: kv[1], 
                             reverse = True)
library_sorted_freq[1:25]

[('https', 604),
 ('to', 389),
 ('of', 349),
 ('and', 339),
 ('a', 249),
 ('for', 248),
 ('in', 190),
 ('ucsb', 184),
 ('our', 166),
 ('library', 161),
 ('s', 157),
 ('at', 132),
 ('you', 126),
 ('on', 125),
 ('from', 115),
 ('more', 111),
 ('this', 110),
 ('we', 105),
 ('is', 101),
 ('with', 101),
 ('by', 101),
 ('here', 95),
 ('amp', 94),
 ('are', 78)]

We can get the english stopwords out.

In [51]:
# load the stopwords to use
from nltk.corpus import stopwords
sw_nltk = stopwords.words('english')

In [52]:
# create a new text list that does
# NOT contain stopwords
library_str_stopped = [word for word in library_string.split() 
                       if word.lower() not in sw_nltk]
library_words_stopped = " ".join(library_str_stopped)

The output with the stopwords removed is a little better, but there's still some cruft that could be removed:

In [53]:
library_blob_stopped = textblob.TextBlob(library_words_stopped)
library_blob_stopped_freq = library_blob_stopped.word_counts
library_blob_stopped_sorted_freq = sorted(library_blob_stopped_freq.items(), 
                             key = lambda kv: kv[1], 
                             reverse = True)
library_blob_stopped_sorted_freq[1:50]

[('ucsb', 184),
 ('library', 161),
 ('s', 146),
 ('amp', 94),
 ('here', 86),
 ('’', 67),
 ('ucsblibrary', 63),
 ('research', 60),
 ('us', 57),
 ('uc', 56),
 ('today', 55),
 ('join', 55),
 ('book', 50),
 ('open', 49),
 ('new', 48),
 ('reads', 47),
 ('learn', 44),
 ('more', 43),
 ('week', 43),
 ('we', 40),
 ('access', 39),
 ('students', 38),
 ('nhttps', 34),
 ('tomorrow', 34),
 ('study', 34),
 ('community', 33),
 ('2022', 33),
 ('event', 33),
 ('campus', 33),
 ('info', 32),
 ('santa', 32),
 ('exhibit', 32),
 ('recordings', 31),
 ('barbara', 31),
 ('re', 31),
 ('art', 30),
 ('check', 29),
 ('collections', 27),
 ('resources', 27),
 ('day', 27),
 ('talk', 26),
 ('ted', 26),
 ('exhalation', 26),
 ('collection', 26),
 ('one', 26),
 ('black', 26),
 ('chiang', 25),
 ('discussion', 25),
 ('register', 25)]

In [54]:
# a more meaningful segment
library_blob_stopped_sorted_freq[7:57]

[('ucsblibrary', 63),
 ('research', 60),
 ('us', 57),
 ('uc', 56),
 ('today', 55),
 ('join', 55),
 ('book', 50),
 ('open', 49),
 ('new', 48),
 ('reads', 47),
 ('learn', 44),
 ('more', 43),
 ('week', 43),
 ('we', 40),
 ('access', 39),
 ('students', 38),
 ('nhttps', 34),
 ('tomorrow', 34),
 ('study', 34),
 ('community', 33),
 ('2022', 33),
 ('event', 33),
 ('campus', 33),
 ('info', 32),
 ('santa', 32),
 ('exhibit', 32),
 ('recordings', 31),
 ('barbara', 31),
 ('re', 31),
 ('art', 30),
 ('check', 29),
 ('collections', 27),
 ('resources', 27),
 ('day', 27),
 ('talk', 26),
 ('ted', 26),
 ('exhalation', 26),
 ('collection', 26),
 ('one', 26),
 ('black', 26),
 ('chiang', 25),
 ('discussion', 25),
 ('register', 25),
 ('online', 25),
 ('history', 24),
 ('pick', 24),
 ('free', 23),
 ('librarian', 22),
 ('work', 22),
 ('year', 22)]

In [55]:
# Challenge: for the Python wizzes. #FIXME
# do that in a tidy way?
# what do pandas pipes look like?

# Just the words, hold the gore

## Challenge: Insta-rrectionists

In [56]:
# how long is this?
riots_dehydrated_df = pandas.read_csv("raw_data/dehydratedCapitolRiotTweets.txt")
len(riots_dehydrated_df)

82308

In [57]:
# this takes a very long time.
# !twarc2 hydrate raw_data/dehydratedCapitolRiotTweets.txt raw_data/riots.jsonl

In [58]:
# regardless of how you slice this, it's about 
# 80 % of the content that is still there.

# these are very slow, so they are commented out
# ! twarc2 flatten raw_data/riots.jsonl output_data/riots_flat.jsonl
# ! twarc2 csv output_data/riots_flat.jsonl output_data/riots_flat.csv

! wc output_data/riots_flat.jsonl
! wc output_data/riots_flat.csv


    65554  31279893 553455111 output_data/riots_flat.jsonl
    65322  12010592 360330969 output_data/riots_flat.csv


In [59]:
# let's deal with just 10,000 of these chuckleheads:
! head -n 10000 output_data/riots_flat.jsonl > output_data/riots10k.jsonl
! twarc2 csv output_data/riots10k.jsonl > output_data/riots10k.csv

In [60]:
 riots_df = pandas.read_csv("output_data/riots10k.csv", low_memory=False)

In [61]:
riots_df.shape

(9997, 74)

In [62]:
riots_df.columns

Index(['id', 'conversation_id', 'referenced_tweets.replied_to.id',
       'referenced_tweets.retweeted.id', 'referenced_tweets.quoted.id',
       'author_id', 'in_reply_to_user_id', 'retweeted_user_id',
       'quoted_user_id', 'created_at', 'text', 'lang', 'source',
       'public_metrics.like_count', 'public_metrics.quote_count',
       'public_metrics.reply_count', 'public_metrics.retweet_count',
       'reply_settings', 'possibly_sensitive', 'withheld.scope',
       'withheld.copyright', 'withheld.country_codes', 'entities.annotations',
       'entities.cashtags', 'entities.hashtags', 'entities.mentions',
       'entities.urls', 'context_annotations', 'attachments.media',
       'attachments.media_keys', 'attachments.poll.duration_minutes',
       'attachments.poll.end_datetime', 'attachments.poll.id',
       'attachments.poll.options', 'attachments.poll.voting_status',
       'attachments.poll_ids', 'author.id', 'author.created_at',
       'author.username', 'author.name', 'author

In [108]:
# count the users
unique_users_df = riots_df.author_id.unique()
(unique_users_df.shape)

(9160,)

In [109]:
# I'm forgetting what this really does for us. The 9160 unique
# authors quoted 230 different other people?
users_quoted_df = riots_df.quoted_user_id.unique()
(users_quoted_df.shape)

(230,)

# Episode 6: Search and Filter

In [65]:
# use Twitter advanced search syntax (everthing in quotes!)
# to get tailored results
# !twarc2 search --limit 800 "(cute OR fluffy OR haircut) (#catsofinstagram) lang:en" raw_data/kittens.jsonl
!twarc2 csv raw_data/kittens.jsonl output_data/kittens.csv

100%|██████████████| Processed 2.59M/2.59M of input file [00:00<00:00, 3.44MB/s]

ℹ️
Parsed 899 tweets objects from 9 lines in the input file.
Wrote 899 rows and output 74 columns in the CSV.



In [66]:
kittens_df = pandas.read_csv("output_data/kittens.csv")

In [67]:
kittens_df

,id,conversation_id,referenced_tweets.replied_to.id,referenced_tweets.retweeted.id,referenced_tweets.quoted.id,author_id,in_reply_to_user_id,retweeted_user_id,quoted_user_id,created_at,...,geo.geo.bbox,geo.geo.type,geo.id,geo.name,geo.place_id,geo.place_type,__twarc.retrieved_at,__twarc.url,__twarc.version,Unnamed: 73
0,1528574826507554816,1528574826507554816,NaN,1.526889e+18,NaN,1339747029451419648,NaN,3.958303e+09,NaN,2022-05-23T03:13:32.000Z,...,NaN,NaN,NaN,NaN,NaN,NaN,2022-05-23T03:17:48+00:00,https://api.twitter.com/2/tweets/search/recent...,2.10.4,NaN
1,1528570565065379840,1528570565065379840,NaN,1.526889e+18,NaN,1052848710,NaN,3.958303e+09,NaN,2022-05-23T02:56:36.000Z,...,NaN,NaN,NaN,NaN,NaN,NaN,2022-05-23T03:17:48+00:00,https://api.twitter.com/2/tweets/search/recent...,2.10.4,NaN
2,1528558400346259456,1528558400346259456,NaN,1.528045e+18,NaN,616608467,NaN,1.212552e+18,NaN,2022-05-23T02:08:16.000Z,...,NaN,NaN,NaN,NaN,NaN,NaN,2022-05-23T03:17:48+00:00,https://api.twitter.com/2/tweets/search/recent...,2.10.4,NaN
3,1528547008151101440,1528547008151101440,NaN,1.528454e+18,NaN,1373558094693683204,NaN,2.926437e+09,NaN,2022-05-23T01:23:00.000Z,...,NaN,NaN,NaN,NaN,NaN,NaN,2022-05-23T03:17:48+00:00,https://api.twitter.com/2/tweets/search/recent...,2.10.4,NaN
4,1528544995975757827,1528544995975757827,NaN,1.528153e+18,NaN,264383293,NaN,2.826013e+09,NaN,2022-05-23T01:15:00.000Z,...,NaN,NaN,NaN,NaN,NaN,NaN,2022-05-23T03:17:48+00:00,https://api.twitter.com/2/tweets/search/recent...,2.10.4,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
894,1526927292382969858,1526927292382969858,NaN,1.526889e+18,NaN,1507717680421298181,NaN,3.958303e+09,NaN,2022-05-18T14:06:50.000Z,...,NaN,NaN,NaN,NaN,NaN,NaN,2022-05-23T03:17:55+00:00,https://api.twitter.com/2/tweets/search/recent...,2.10.4,NaN
895,1526927183905693696,1526927183905693696,NaN,1.526889e+18,NaN,862826271016931331,NaN,3.958303e+09,NaN,2022-05-18T14:06:24.000Z,...,NaN,NaN,NaN,NaN,NaN,NaN,2022-05-23T03:17:55+00:00,https://api.twitter.com/2/tweets/search/recent...,2.10.4,NaN
896,1526926332516515841,1526926332516515841,NaN,1.526889e+18,NaN,373612360,NaN,3.958303e+09,NaN,2022-05-18T14:03:01.000Z,...,NaN,NaN,NaN,NaN,NaN,NaN,2022-05-23T03:17:55+00:00,https://api.twitter.com/2/tweets/search/recent...,2.10.4,NaN
897,1526926256259973121,1526926256259973121,NaN,1.526889e+18,NaN,1232696931408924677,NaN,3.958303e+09,NaN,2022-05-18T14:02:43.000Z,...,NaN,NaN,NaN,NaN,NaN,NaN,2022-05-23T03:17:55+00:00,https://api.twitter.com/2/tweets/search/recent...,2.10.4,NaN


In [68]:
list(kittens_df.columns)

['id',
 'conversation_id',
 'referenced_tweets.replied_to.id',
 'referenced_tweets.retweeted.id',
 'referenced_tweets.quoted.id',
 'author_id',
 'in_reply_to_user_id',
 'retweeted_user_id',
 'quoted_user_id',
 'created_at',
 'text',
 'lang',
 'source',
 'public_metrics.like_count',
 'public_metrics.quote_count',
 'public_metrics.reply_count',
 'public_metrics.retweet_count',
 'reply_settings',
 'possibly_sensitive',
 'withheld.scope',
 'withheld.copyright',
 'withheld.country_codes',
 'entities.annotations',
 'entities.cashtags',
 'entities.hashtags',
 'entities.mentions',
 'entities.urls',
 'context_annotations',
 'attachments.media',
 'attachments.media_keys',
 'attachments.poll.duration_minutes',
 'attachments.poll.end_datetime',
 'attachments.poll.id',
 'attachments.poll.options',
 'attachments.poll.voting_status',
 'attachments.poll_ids',
 'author.id',
 'author.created_at',
 'author.username',
 'author.name',
 'author.description',
 'author.entities.description.cashtags',
 'author

# Search

Twitter search using Boolean logic 

### Using And 
In twarc, a space between operators will act as `AND`

In [69]:

!twarc2 search --limit 200 "grumpy cat" > raw_data/grumpy_throwaway.jsonl
# this will return tweets matching both conditions 
# running without a limit will crash the kernel the cats are so strong 

### Using Or

In twarc, using OR will return results where either condition is met 

In [70]:
!twarc2 search  --limit 200 "grumpy OR cat" > raw_data/grumpy_throwaway.jsonl

#this will return tweets where either search condition is met, grumpy or cat 
#running without a limit will crash the kernel grumpy people and cats are so strong 

### Not Logic 

What about negating certain terms within a search? Use a dash (-) followed by the keyword you want to avoid

In [71]:
# lets ignore Doja Cat from our internet cat search

!twarc2 search --limit 200 "grumpy OR cat -Dojacat" > raw_data/grumpy_throwaway.jsonl

### Searching by User 

Looking for mentions, tos/froms 

In [72]:
#to: will match any tweeet that is a reply to a particular user 
#You can only pass a single username per to: command 
!twarc2 search "(to:realgrumpycat)" > raw_data/grumpy_throwaway.jsonl

Likewise, from: will match any tweet from a specific user 

In [73]:
 !twarc2 search "(from:realgrumpycat)" > raw_data/grumpy_throwaway.jsonl

### Using Hashtags

In [74]:
# the below command would be a good way to blow your quota
# !twarc2 search "#meme" > raw_data/grumpy_throwaway.jsonl

### Order of Operations 

Of course, you can combine search queries to narrow down results but remember these rules. 
You can also use () to group terms to eliminate any uncertainty

* cats OR cole marmalade > cats OR (cole marmalade) 
* cats cole OR marmalade > (cats cole) OR marmalade

* (cats or cole) marmalade > some cats, some guys named Cole, and orange preservatives 
* cat (cole marmalade) > group them together, and give us the actual cat duo 

### Advanced Cat Challenge 

In [75]:
# !twarc2 search "(grumpy cat #meme)"
 
# !twarc2 search "(grumpy cat) OR (#meme has:images)"

# !twarc2 search "(cats OR puppies) has:media"

# !twarc2 search "(to:_We_Rate_Cats) lang:en"

# Stream

In [76]:
!twarc2 stream-rules add "#WorldGothDay"

🚀  Added rule for "#WorldGothDay"


In [77]:
!twarc2 stream-rules add "gothcats"

🚀  Added rule for "gothcats"


In [78]:
# press the square to interrupt this!

In [79]:
# !twarc2 stream > "raw_data/streamed_goth.jsonl"

In [80]:
! wc raw_data/streamed_goth.jsonl
! twarc2 flatten raw_data/streamed_goth.jsonl > output_data/streamed_goth_flat.jsonl
! wc output_data/streamed_goth_flat.jsonl 

    4  1277 20136 raw_data/streamed_goth.jsonl
    4  1582 24554 output_data/streamed_goth_flat.jsonl


In [81]:
!twarc2 stream-rules delete "gothcats"
!twarc2 stream-rules delete "#WorldGothDay"

🗑  Deleted stream rule for gothcats
🗑  Deleted stream rule for #WorldGothDay


# Episode 7: twarc plug-ins

### install the plug-ins
you'll need to do this each time your kernel restarts

In [ ]:
!pip install twarc-hashtags

In [ ]:
!pip install twarc-network

In [ ]:
# retweeted-by is a built-in command. no plug-in necessary. It takes a tweet ID:

In [88]:
!twarc2 retweeted-by 1522543998996414464 > 'raw_data/tinycarebot_rtby.jsonl'
!twarc2 flatten raw_data/tinycarebot_rtby.jsonl > output_data/tinycarebot_rtby_flat.jsonl

Speaking of retweeting, it's very good to figure out how much of your dataset 
is tweets, and how much of it is retweets and quotes.

# Retweets vs. tweets
How much original content is there?
Do this for both library timeline and catsofinstagrams

In [82]:
# via pandas and plottting
retweet_count = hashtagcats_df["referenced_tweets.retweeted.id"].value_counts()
sum(retweet_count)


413

In [83]:
(sum(retweet_count) / len(hashtagcats_df))

0.6894824707846411

69% of the tweets that used #catsofinstagram were retweets.

In [84]:
# so our pipeline on a stream would look like:


### Followers

In [92]:
# this is slow and uses quota
# that's 1000 followers.
# !twarc2 followers --limit 1 tinycarebot >  'raw_data/tcb_followers.jsonl'

In [93]:
!twarc2 flatten raw_data/tcb_followers.jsonl > output_data/tcb_followers_flat.jsonl

In [94]:
! wc output_data/tcb_followers_flat.jsonl

   1000   54050 1212647 output_data/tcb_followers_flat.jsonl


In [95]:
# tiny challenge: do robots follow robots?
# look at the help!
! twarc2 csv --input-data-type users output_data/tcb_followers_flat.jsonl > output_data/tcb_followers.csv
# csv doesn't work on profiles.

### Most used hashtags

In [113]:
!twarc2 hashtags raw_data/hashtagcats.jsonl output_data/hashtagcats_hashtags.csv

100%|██████████████| Processed 1.84M/1.84M of input file [00:00<00:00, 19.7MB/s]


In [91]:
!twarc2 network raw_data/hashtagcats.jsonl output_data/hashtagcats_network.html

In [97]:
!twarc2 network raw_data/hashtag_gasprices.jsonl output_data/hashtag_gasprices_network.html

In [90]:
# how do I print file to cell?
# print(read(output_data/hashtagcats_hashtags.csv))

In [85]:
# this reminds you what DataFrames you have in memory
%who DataFrame

ecodatasci_df	 hashtagcats_df	 kittens_df	 library_timeline_df	 most_rt	 riots_dehydrated_df	 riots_df	 sort_by_rt	 ucsblib_timeline_df	 



In [96]:
# when do we do mentions?
# ! twarc2 mentions ucsblibrary raw_data/ucsblibrary_mentions.jsonl
# what comes next now that users csv is fixed!????? #FIXME
# ! twarc2 csv raw_data/ucsblibrary_mentions.jsonl output_data/ucsblibrary_mentions.csv 
# ucsb_library_mentions_df = pandas.read_csv('output_data/ucsblibrary_mentions.csv')

# Episode 8: Python text analysis

### Sentiment Analysis
To do this, we need to do a little Python

TextBlob is a text processing library that does sentiment analysis. 
The sentiment property returns a namedtuple of the form Sentiment(polarity, subjectivity). The polarity score is a float within the range [-1.0, 1.0]. The subjectivity is a float within the range [0.0, 1.0] where 0.0 is very objective and 1.0 is very subjective.

Before we use TextBlob for sentiment analysis, we need to download
datasets of words and their associated weights. These are called *corpora*.

In [98]:
# commented out because I put it up in ep 2
# !python -m textblob.download_corpora

In [99]:
# TextBlob needs a string, so this won't work.
# textblob.TextBlob(hashtagcats_df).sentiment

In [100]:
# even calling the column won't work:
# textblob.TextBlob(hashtagcats_df['text']).sentiment

In [101]:
# break tweets test column into a list, then .join into one long string 
hashtagcats_list = ' '.join(hashtagcats_df['text'].tolist())
# turn the string into a blob
hashtagcats_blob = textblob.TextBlob(hashtagcats_list)
# get the sentiment
hashtagcats_blob.sentiment

Sentiment(polarity=0.17576840733247795, subjectivity=0.5069781628198716)

In [102]:
# what dataframes are still here?
%whos DataFrame


Variable              Type         Data/Info
--------------------------------------------
ecodatasci_df         DataFrame                          id <...>\n[473 rows x 74 columns]
hashtagcats_df        DataFrame                          id <...>\n[599 rows x 74 columns]
kittens_df            DataFrame                          id <...>\n[899 rows x 74 columns]
library_timeline_df   DataFrame                          id <...>\n[500 rows x 74 columns]
most_rt               DataFrame                          id <...>\n\n[1 rows x 74 columns]
riots_dehydrated_df   DataFrame           134686307243517952<...>n[82308 rows x 1 columns]
riots_df              DataFrame                           id<...>n[9997 rows x 74 columns]
sort_by_rt            DataFrame                          id <...>\n[500 rows x 74 columns]
ucsblib_timeline_df   DataFrame                          id <...>\n[500 rows x 74 columns]


The overall sentiment of the language of kitty twitter is rather positive.
And the tweets tend to be subjective.

In [103]:
# What do you think the sentiment of gasprices might be?
# get the overall sentiment and see if it matches your prediction.

In [104]:
! twarc2 csv output_data/hashtag_gasprices_flat.jsonl > output_data/hashtag_gasprices_flat.csv
hashtag_gasprices_df = pandas.read_csv("output_data/hashtag_gasprices_flat.csv", low_memory=False)

In [105]:
gasprices_list = ' '.join(hashtag_gasprices_df['text'].tolist())
gasprices_blob = textblob.TextBlob(gasprices_list)
print("Hashtag Gas Prices: ") 
gasprices_blob.sentiment

Hashtag Gas Prices: 


Sentiment(polarity=0.0783091422742766, subjectivity=0.44952337845641754)

In [106]:
hashtagcats_list = ' '.join(hashtagcats_df['text'].tolist())
hashtagcats_blob = textblob.TextBlob(hashtagcats_list)
print("Hashtag Cats of Instagram: ") 
hashtagcats_blob.sentiment

Hashtag Cats of Instagram: 


Sentiment(polarity=0.17576840733247795, subjectivity=0.5069781628198716)

# Episode 9: Data Management

In [107]:
!twarc2 counts --granularity "day" --text "amazon"

2022-05-18T19:27:23.000Z - 2022-05-19T00:00:00.000Z: 155,378
2022-05-19T00:00:00.000Z - 2022-05-20T00:00:00.000Z: 901,716
2022-05-20T00:00:00.000Z - 2022-05-21T00:00:00.000Z: 997,120
2022-05-21T00:00:00.000Z - 2022-05-22T00:00:00.000Z: 886,295
2022-05-22T00:00:00.000Z - 2022-05-23T00:00:00.000Z: 879,216
2022-05-23T00:00:00.000Z - 2022-05-24T00:00:00.000Z: 958,872
2022-05-24T00:00:00.000Z - 2022-05-25T00:00:00.000Z: 1,034,211
2022-05-25T00:00:00.000Z - 2022-05-25T19:27:23.000Z: 849,022

Total Tweets: 6,661,830

